# Benchmark Clustering

This notebook compares the relative speed and accuracy of the different clustering algorithms. It can be used both to demonstrate how those algorithms work and to compare their performance. See the next section for the definition of the various parameters that we would expect to impact the performance of the clustering.

**Note:** The notebook inserts fake **linear** trajectories, so it will only indicate the performance of clustering those. For objects whose movement is very nonlinear, we might see better performance from other clustering algorithms. However, since KBMOD searches for linear trajectories, it makes sense to evaluate the clustering algorithm relative to those.

In [ ]:
from astropy.table import Table
import numpy as np

from kbmod.configuration import SearchConfiguration
from kbmod.fake_data.fake_data_creator import create_fake_times, FakeDataSet
from kbmod.filters.clustering_filters import apply_clustering
from kbmod.results import Results
from kbmod.run_search import SearchRunner
from kbmod.search import Trajectory
from kbmod.trajectory_generator import VelocityGridSearch
from kbmod.trajectory_utils import match_trajectory_sets

import cProfile
import timeit

rng = np.random.default_rng()

## Define critical parameters

We predefine a few parameters that we expect to have a large impact on the performance of the clustering algorithm. Users may want to vary these to determine their impact.

In [ ]:
# Data set sizes.  Larger image sizes will mean more potential results found (including noise).
num_times = 20
width = 600
height = 500

# Number of trajectories to insert.
num_trjs = 20

# The number of results returned per pixel.
results_per_pixel = 3

## Create a fake set of images with inserted trajectories

We create a data set that initially consists of empty, noisy images. We sample 5 observations per night (spaced by ~30 minutes) on 4 consecutive nights for a total of 20 images.

In [ ]:
# Create fake times with 5 observations per night and 20 total.
times = create_fake_times(
    num_times, t0=0.0, obs_per_day=5, intra_night_gap=0.02, inter_night_gap=1
)

# Create a fake data set.
psf_val = 1.0
fake_ds = FakeDataSet(
    width, height, times, noise_level=1.0, psf_val=psf_val, use_seed=True
)

Create a series of fake trajectories. Since we want these all to be detectable, we keep them to the left half (x <= width/2) and the middle of the chip (height/4 <= y <= 3*height/4). We then set the velocities so vx is always positive and vy can be positive or negative. Each true trajectory is inserted into each image and saved to a list for later comparison.

In [ ]:
print(f"Sampling x from [0, {int(width/2)}]")
print(f"Sampling y from [{int(height/4)}, {int(3*height/4)}]")

all_trjs = []
for i in range(num_trjs):
    trj = Trajectory(
        x=int(width / 2 * rng.random()),
        y=int(height / 2 * rng.random() + height / 4),
        vx=20.0 * rng.random(),
        vy=20.0 * rng.random() - 10.0,
        flux=200,
    )
    print(f"Trajectory {i}: {trj}")

    # Insert the object into the images.
    fake_ds.insert_object(trj)

    # Save the trajectory to the list.
    all_trjs.append(trj)

## Perform the search

The search itself will use a grid of velocities where x=[0.0, 20.0] in 41 steps and y=[-10.0, 10.0] in 41 steps for a total of 1681 candidates per pixel.  The starting x and y locations for the search include all pixels in the images.

Very minimal filtering is done to remove trajectories that are not visible in at least half the observations (`min_obs` parameter) or have negative likleihood (`lh_level` parameter). We also increase the maximum likelihood threshold. GPU filtering and masking are both turned off.

In [ ]:
min_obs = int(num_times / 2)

input_parameters = {
    "chunk_size": 10_000_000,
    "do_mask": False,
    "gpu_filter": False,
    "lh_level": 0.00000001,
    "max_lh": 100000.0,
    "num_obs": min_obs,
    "psf_val": psf_val,
    "results_per_pixel": results_per_pixel,
    "sigmaG_lims": [25, 75],
}
config = SearchConfiguration.from_dict(input_parameters)

# Create the search trajectories.
trj_generator = VelocityGridSearch(41, 0.0, 20.0, 41, -10.0, 10.0)
candidates = [trj for trj in trj_generator]
print(f"Testing {len(candidates)} trajectories per pixel.")

# Do the actual search.
search = SearchRunner()
results = search.do_gpu_search(config, fake_ds.stack, trj_generator)
print(f"Ran search and generated {len(results)} results.")

For each of the trajectories inserted (true fakes), find the closest matching result that is no more than 5.0 pixels away on average.

In [ ]:
def _compute_match_stats(all_trjs, results, threshold, times):
    found_trjs = results.make_trajectory_list()
    all_matches = match_trajectory_sets(all_trjs, found_trjs, threshold, times=times)
    num_found = np.count_nonzero(all_matches > -1)
    num_missed = len(all_trjs) - num_found
    return num_found, num_missed


num_found, num_missed = _compute_match_stats(all_trjs, results, 5.0, times)
print(
    f"Matched {num_found} (and missed {num_missed}) of {len(all_trjs)} inserted trajectories."
)

## Run and evaluate clustering algorithms

Iterate through the different clustering algorithms along with their threshold parameter. Each of these clustering calls takes a while, so we print out progress markers for each run.

**Note:** Timing only uses a single clustering run (instead of the average of a bunch), so it will be noisy.

In [ ]:
# Create the clustering parameters
cluster_params = {
    "cluster_type": "all",
    "cluster_eps": 20.0,
    "cluster_v_scale": 1.0,
    "times": np.array(times),
}

type_vals = [
    "all",
    "position",
    "mid_position",
    "start_end_position",
    "nn_start_end",
    "nn_start",
]
cluster_eps_vals = [1.0, 2.0, 5.0, 10.0, 20.0]

stats_dict = {
    "name": [],
    "eps": [],
    "total": [],
    "num_found": [],
    "num_missed": [],
    "run_time": [],
}
for type_name in type_vals:
    for eps in cluster_eps_vals:
        print(f"Testing '{type_name}': {eps}")
        cluster_params["cluster_type"] = type_name
        cluster_params["cluster_eps"] = eps

        # Make a temporary copy of the results for filtering.
        tmp_res = results.copy()

        # Do the clustering.
        run_time = timeit.timeit(
            "apply_clustering(tmp_res, cluster_params)", number=1, globals=globals()
        )

        # Score the results.
        num_found, num_missed = _compute_match_stats(all_trjs, tmp_res, 5.0, times)
        stats_dict["name"].append(type_name)
        stats_dict["eps"].append(eps)
        stats_dict["total"].append(len(tmp_res))
        stats_dict["num_found"].append(num_found)
        stats_dict["num_missed"].append(num_missed)
        stats_dict["run_time"].append(run_time)

In [ ]:
# Print the table out in blocks by clustering algorithm.
tbl = Table(stats_dict)
for type_name in type_vals:
    print(f"\nData for clustering={type_name}")
    print(tbl[tbl["name"] == type_name])